# Script Set Up

In [ ]:
%matplotlib inline

In [ ]:
!pip install mne

In [ ]:
!pip install pingouin
import pingouin as pg

In [ ]:
!pip install dataframe-image
import dataframe_image as dfi

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns
import mne

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Deciding what response times are outliers that should be removed

In [ ]:
lower_RT_cutoff = 150
upper_RT_cutoff = 1500 # 1.5x longest SI and is less than 1% of trials

# Load Epochs
Load each participant's epochs

In [ ]:
ID_list = ['P1', 'P3', 'P4', 'P7', 'P10', 'P12', 'P15', 'P17', 'P19', 'P21', 'P22', 'P23', 'P24',
           'P26', 'P27', 'P28', 'P29', 'P30']

n_participants = len(ID_list)
n_channels_evoked = 128
n_channels_epochs = 145
n_times_encode = 717
n_times_repo = 616
n_times_RL = 615

sample_rate = 512

encode_epochs = []
encode_metadata = []
repo_epochs = []
repo_metadata = []
RL_epochs = []
RL_metadata = []

all_encode_evokeds = np.zeros([len(ID_list), n_channels_evoked, n_times_encode])
all_repo_evokeds = np.zeros([len(ID_list), n_channels_evoked, n_times_repo])
all_RL_evokeds = np.zeros([len(ID_list), n_channels_evoked, n_times_RL])

i = -1

for ID in ID_list:
  i += 1
  encode_fname = "/content/gdrive/MyDrive/Time_Reproduction/Epochs_CSD/Encode/" + ID + "_Encode_All_CSD_epo.fif"
  repo_fname = "/content/gdrive/MyDrive/Time_Reproduction/Epochs_CSD/Reproduction/" + ID + "_Reproduction_All_CSD_epo.fif"
  RL_fname = "/content/gdrive/MyDrive/Time_Reproduction/Epochs_CSD/Response_Locked/" + ID + "_RL_All_CSD_epo.fif"

  encode_epochs_participant = mne.read_epochs(encode_fname, preload = False)
  repo_epochs_participant = mne.read_epochs(repo_fname, preload = False)
  RL_epochs_participant = mne.read_epochs(RL_fname, preload = False)

  encode_epochs_participant.load_data()
  repo_epochs_participant.load_data()
  RL_epochs_participant.load_data()

  info = encode_epochs_participant.info

  # Adding ID column to metadata 19/12/22
  encode_df = encode_epochs_participant.metadata
  encode_df["ID"] = ID # adding ID column
  temp_cols=encode_df.columns.tolist()
  new_cols=temp_cols[-1:] + temp_cols[:-1]
  encode_df=encode_df[new_cols]
  encode_metadata.append(encode_df)

  repo_df = repo_epochs_participant.metadata
  repo_df["ID"] = ID # adding ID column
  temp_cols=repo_df.columns.tolist()
  new_cols=temp_cols[-1:] + temp_cols[:-1]
  repo_df=repo_df[new_cols]
  repo_metadata.append(repo_df)

  RL_df = RL_epochs_participant.metadata
  RL_df["ID"] = ID # adding ID column
  temp_cols=RL_df.columns.tolist()
  new_cols=temp_cols[-1:] + temp_cols[:-1]
  RL_df=RL_df[new_cols]
  RL_metadata.append(RL_df)

  ## Evoked ##
  all_encode_evokeds[i] = encode_epochs_participant.average().data
  all_repo_evokeds[i] = repo_epochs_participant.average().data
  all_RL_evokeds[i] = RL_epochs_participant.average().data

# Concatenate Metadata
Concatenate metadata so that all participants epochs are in one array


In [ ]:
encode_metadata = pd.concat(encode_metadata, ignore_index = True)
encode_metadata

In [ ]:
repo_metadata = pd.concat(repo_metadata, ignore_index = True)
repo_metadata

In [ ]:
RL_metadata = pd.concat(RL_metadata, ignore_index = True)
RL_metadata

# Defining X-axis and Color Palette for Plots
For the waveform plots, there is 1 data point for each time point (i.e each sample), so the x-axis corresponds to the number of time points in an epoch

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
encode_times_for_plot = (np.arange(0, n_times_encode) / 512 * 1000) - 200
repo_times_for_plot = (np.arange(0, n_times_repo) / 512 * 1000) - 300
RL_times_for_plot = (np.arange(0, n_times_RL) / 512 * 1000) - 1100

# Define Time Windows for Calculations


In [ ]:
# For checking amplitude at target offset
encode_amplitude_window_start = -50
encode_amplitude_window_end = +50 # these have to be done within sample interval bin

# For checking the amplitude at cue onset
repo_amplitude_window_start = -50
repo_amplitude_window_start_idx = int(np.argwhere(repo_times_for_plot >= repo_amplitude_window_start)[0])
repo_amplitude_window_end = +50
repo_amplitude_window_end_idx = int(np.argwhere(repo_times_for_plot >= repo_amplitude_window_end)[0])

# For checking build-up rate in anticipation of response
RL_slope_window_start = -600
RL_slope_window_start_idx = int(np.argwhere(RL_times_for_plot >= RL_slope_window_start)[0])
RL_slope_window_end = -300
RL_slope_window_end_idx = int(np.argwhere(RL_times_for_plot >= RL_slope_window_end)[0])

# For checking amp of pre-response CNV
RL_amp_window_start = -100
RL_amp_window_start_idx = int(np.argwhere(RL_times_for_plot >= RL_amp_window_start)[0])
RL_amp_window_end = -50
RL_amp_window_end_idx = int(np.argwhere(RL_times_for_plot >= RL_amp_window_end)[0])

# For checking build-up rate of CPP in anticipation of response
RL_CPP_window_start = -600
RL_CPP_window_start_idx = int(np.argwhere(RL_times_for_plot >= RL_CPP_window_start)[0])
RL_CPP_window_end = -300
RL_CPP_window_end_idx = int(np.argwhere(RL_times_for_plot >= RL_CPP_window_end)[0])

# Plotting Grand Average

## Average Across Participants and Convert into MNE Object

In [ ]:
group_encode_evoked_data = np.mean(all_encode_evokeds, axis = 0)
group_repo_evoked_data = np.mean(all_repo_evokeds, axis = 0)
group_RL_evoked_data = np.mean(all_RL_evokeds, axis = 0)

print(group_encode_evoked_data.shape)
print(group_repo_evoked_data.shape)
print(group_RL_evoked_data.shape)

group_encode_evoked = mne.EvokedArray(group_encode_evoked_data, info = info, tmin = -0.2, nave = len(ID_list))
group_repo_evoked = mne.EvokedArray(group_repo_evoked_data, info = info, tmin = -0.1, nave = len(ID_list))
group_RL_evoked = mne.EvokedArray(group_RL_evoked_data, info = info, tmin = -1.1, nave = len(ID_list))

print(group_encode_evoked)
print(group_repo_evoked)
print(group_RL_evoked)

In [ ]:
CPP_electrodes = ["A4", "A19"]
CNV_electrodes = ["C23", "C22"]

In [ ]:
CNV_idx = []
for ch in CNV_electrodes:
  CNV_idx.append(group_encode_evoked.ch_names.index(ch))
print(CNV_idx)

CPP_idx = []
for ch in CPP_electrodes:
  CPP_idx.append(group_encode_evoked.ch_names.index(ch))
print(CPP_idx)

# Calculating Single Trial Measurements

## Doing Calculation

In [ ]:
m_idx_encode = -1
m_idx_repo = -1
m_idx_RL = -1

for i in range(0, len(ID_list)):
  ## Loading in epochs and removing outliers
  ID = ID_list[i]
  encode_fname = "/content/gdrive/MyDrive/Time_Reproduction/Epochs_CSD/Encode/" + ID + "_Encode_All_CSD_epo.fif"
  repo_fname = "/content/gdrive/MyDrive/Time_Reproduction/Epochs_CSD/Reproduction/" + ID + "_Reproduction_All_CSD_epo.fif"
  RL_fname = "/content/gdrive/MyDrive/Time_Reproduction/Epochs_CSD/Response_Locked/" + ID + "_RL_All_CSD_epo.fif"

  encode_epochs_participant = mne.read_epochs(encode_fname, preload = False)
  repo_epochs_participant = mne.read_epochs(repo_fname, preload = False)
  RL_epochs_participant = mne.read_epochs(RL_fname, preload = False)

  encode_epochs_participant.load_data()
  repo_epochs_participant.load_data()
  RL_epochs_participant.load_data()

  info = encode_epochs_participant.info

  ## Iterating through each trial for each epoch and doing the calculation ##

  ## Encode Epoch
  encode_epochs_participant_data = encode_epochs_participant.get_data()
  for e in range(0, encode_epochs_participant_data.shape[0]):
    m_idx_encode += 1 # index we are using for adding these measures to the metadata
    this_trial = encode_epochs_participant_data[e] # get data for this trial
    CNV_electrodes_this_trial = np.mean(this_trial[CNV_idx], axis = 0) # average across chosen electrodes
    CPP_electrodes_this_trial = np.mean(this_trial[CPP_idx], axis = 0)

    SI = 0 ## doing calculation
    if encode_metadata.at[m_idx_encode, "Trigger"] == 51:
      SI = 600
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 52:
      SI = 650
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 53:
      SI = 700
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 54:
      SI = 750
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 55:
      SI = 800
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 56:
      SI = 850
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 57:
      SI = 900
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 58:
      SI = 950
    elif encode_metadata.at[m_idx_encode, "Trigger"] == 59:
      SI = 1000
    low_cutoff = SI + (encode_amplitude_window_start)
    high_cutoff = SI + (encode_amplitude_window_end)
    encode_window_start_idx = int(np.argwhere(encode_times_for_plot >= low_cutoff)[0])
    encode_window_end_idx = int(np.argwhere(encode_times_for_plot >= high_cutoff)[0])
    encode_metadata.at[m_idx_encode, "Encode_Offset_CNV_Amp"] = np.mean(CNV_electrodes_this_trial[encode_window_start_idx:encode_window_end_idx] * 1e6, axis = 0)
    encode_metadata.at[m_idx_encode, "Encode_Offset_CPP_Amp"] = np.mean(CPP_electrodes_this_trial[encode_window_start_idx:encode_window_end_idx] * 1e6, axis = 0)

    x = encode_times_for_plot[encode_window_start_idx:encode_window_end_idx] # get times for each window as the x-axis
    result = stats.linregress(x,  CNV_electrodes_this_trial[encode_window_start_idx:encode_window_end_idx] * 1e6)
    encode_metadata.at[m_idx_encode, "Encode_Offset_CNV_Slope"] = result.slope
    x = encode_times_for_plot[encode_window_start_idx:encode_window_end_idx] # get times for each window as the x-axis
    result = stats.linregress(x,  CPP_electrodes_this_trial[encode_window_start_idx:encode_window_end_idx] * 1e6)
    encode_metadata.at[m_idx_encode, "Encode_Offset_CPP_Slope"] = result.slope

  ## Repo Epoch
  repo_epochs_participant_data = repo_epochs_participant.get_data()
  for e in range(0, repo_epochs_participant_data.shape[0]):
    m_idx_repo += 1 # index we are using for adding these measures to the metadata
    this_trial = repo_epochs_participant_data[e] # get data for this trial
    CNV_electrodes_this_trial = np.mean(this_trial[CNV_idx], axis = 0) # average across chosen electrodes
    CNV_electrodes_this_trial = CNV_electrodes_this_trial * 1e6
    repo_metadata.at[m_idx_repo, "Repo_Onset_CNV_Amp"] = np.mean(CNV_electrodes_this_trial[repo_amplitude_window_start_idx:repo_amplitude_window_end_idx], axis = 0)

    CPP_electrodes_this_trial = np.mean(this_trial[CPP_idx], axis = 0) # average across chosen electrodes
    CPP_electrodes_this_trial = CPP_electrodes_this_trial * 1e6
    repo_metadata.at[m_idx_repo, "Repo_Onset_CPP_Amp"] = np.mean(CPP_electrodes_this_trial[repo_amplitude_window_start_idx:repo_amplitude_window_end_idx], axis = 0)

  ## RL Epoch
  RL_epochs_participant_data = RL_epochs_participant.get_data()
  for e in range(0, RL_epochs_participant_data.shape[0]):
    m_idx_RL += 1 # index we are using for adding these measures to the metadata
    this_trial = RL_epochs_participant_data[e] # get data for this trial
    CNV_electrodes_this_trial = np.mean(this_trial[CNV_idx], axis = 0) # average across chosen electrodes
    CPP_electrodes_this_trial = np.mean(this_trial[CPP_idx], axis = 0)
    CNV_electrodes_this_trial = CNV_electrodes_this_trial * 1e6
    CPP_electrodes_this_trial = CPP_electrodes_this_trial * 1e6

    # Calculate amplitude
    CNV_electrodes_this_trial = CNV_electrodes_this_trial[RL_amp_window_start_idx: RL_amp_window_end_idx]
    RL_metadata.at[m_idx_RL, "RL_CNV_Amp"] = np.mean(CNV_electrodes_this_trial, axis = 0)

    CPP_electrodes_this_trial = CPP_electrodes_this_trial[RL_CPP_window_start_idx: RL_CPP_window_end_idx]
    RL_metadata.at[m_idx_RL, "RL_CPP_Amp"] = np.mean(CPP_electrodes_this_trial, axis = 0)

    # Calculate slope
    this_trial = RL_epochs_participant_data[e] # get data for this trial
    CNV_electrodes_this_trial = np.mean(this_trial[CNV_idx], axis = 0) # average across chosen electrodes
    CPP_electrodes_this_trial = np.mean(this_trial[CPP_idx], axis = 0)
    CNV_electrodes_this_trial = CNV_electrodes_this_trial * 1e6
    CPP_electrodes_this_trial = CPP_electrodes_this_trial * 1e6
    CNV_electrodes_this_trial = CNV_electrodes_this_trial[RL_slope_window_start_idx: RL_slope_window_end_idx] # get signal in each window as the y-axis
    CPP_electrodes_this_trial = CPP_electrodes_this_trial[RL_CPP_window_start_idx: RL_CPP_window_end_idx]
    x = RL_times_for_plot[RL_slope_window_start_idx: RL_slope_window_end_idx] # get times for each window as the x-axis
    result = stats.linregress(x,  CNV_electrodes_this_trial)
    RL_metadata.at[m_idx_RL, "RL_CNV_Slope"] = result.slope
    x = RL_times_for_plot[RL_CPP_window_start_idx: RL_CPP_window_end_idx] # get times for each window as the x-axis
    result = stats.linregress(x,  CPP_electrodes_this_trial)
    RL_metadata.at[m_idx_RL, "RL_CPP_Slope"] = result.slope

In [ ]:
encode_metadata_copy = encode_metadata.copy()
repo_metadata_copy = repo_metadata.copy()
RL_metadata_copy = RL_metadata.copy()

## Removing Outlier RTs
Removing RTs < 150ms abd RTs > 1500ms

In [ ]:
print(encode_metadata.shape[0])
encode_metadata_outliers = encode_metadata.query("Response_Time < 150").index
print(encode_metadata_outliers)
encode_metadata_outliers = list(encode_metadata_outliers)
print(encode_metadata_outliers)
print(len(encode_metadata_outliers))
encode_metadata_outliers += list(encode_metadata.query("Response_Time > 1500").index)
encode_metadata = encode_metadata.drop(encode_metadata_outliers)
print(encode_metadata.shape[0])

print(repo_metadata.shape[0])
repo_metadata_outliers = repo_metadata.query("Response_Time < 150").index
print(repo_metadata_outliers)
repo_metadata_outliers = list(repo_metadata_outliers)
print(repo_metadata_outliers)
print(len(repo_metadata_outliers))
repo_metadata_outliers += list(repo_metadata.query("Response_Time > 1500").index)
repo_metadata = repo_metadata.drop(repo_metadata_outliers)
print(repo_metadata.shape[0])

print(RL_metadata.shape[0])
RL_metadata_outliers = RL_metadata.query("Response_Time < 150").index
print(RL_metadata_outliers)
RL_metadata_outliers = list(RL_metadata_outliers)
print(RL_metadata_outliers)
print(len(RL_metadata_outliers))
RL_metadata_outliers += list(RL_metadata.query("Response_Time > 1500").index)
RL_metadata = RL_metadata.drop(RL_metadata_outliers)
print(RL_metadata.shape[0])

## Re-Label Trigger Variable as SI

In [ ]:
replace_dict = {51: 600, 52: 650, 53: 700, 54: 750, 55: 800, 56: 850, 57: 900, 58: 950, 59: 1000}
encode_metadata["Trigger"].replace(replace_dict, inplace = True)
encode_metadata.rename(columns = {"Trigger": "SI"}, inplace = True)

replace_dict = {151: 600, 152: 650, 153: 700, 154: 750, 155: 800, 156: 850, 157: 900, 158: 950, 159: 1000}
repo_metadata["Trigger"].replace(replace_dict, inplace = True)
repo_metadata.rename(columns = {"Trigger": "SI"}, inplace = True)

replace_dict = {201: 600, 202: 650, 203: 700, 204: 750, 205: 800, 206: 850, 207: 900, 208: 950, 209: 1000}
RL_metadata["Trigger"].replace(replace_dict, inplace = True)
RL_metadata.rename(columns = {"Trigger": "SI"}, inplace = True)

In [ ]:
SIs = np.unique(RL_metadata["SI"])

# Bin by RT via median split within participant within sample interval

In [ ]:
RT_cutoffs = np.zeros((len(ID_list), len(SIs)))
repo_RT_cutoffs = np.zeros((len(ID_list), len(SIs)))
encode_RT_cutoffs = np.zeros((len(ID_list), len(SIs)))

for i in range(0, len(ID_list)): # iterate through participants
  ID = ID_list[i]
  RT_cutoff_1 = np.zeros(len(SIs))
  repo_RT_cutoff_1 = np.zeros(len(SIs))
  encode_RT_cutoff_1 = np.zeros(len(SIs))

  for j in range(0, len(SIs)): # iterate through SIs
    si = SIs[j]
    #RL_participant_df = RL_metadata.query("ID == @ID & SI == @si").copy()
    RL_participant_df = RL_metadata[(RL_metadata["ID"] == ID) & (RL_metadata["SI"] == si)].copy()
    RT_cutoffs[i, j] = RL_participant_df["Response_Time"].median()

    #repo_participant_df = repo_metadata.query("ID == @ID & SI == @si").copy()
    repo_participant_df = repo_metadata[(repo_metadata["ID"] == ID) & (repo_metadata["SI"] == si)].copy()
    repo_RT_cutoffs[i, j] = repo_participant_df["Response_Time"].median()

    #encode_participant_df = encode_metadata.query("ID == @ID & SI == @si").copy()
    encode_participant_df = encode_metadata[(encode_metadata["ID"] == ID) & (encode_metadata["SI"] == si)].copy()
    encode_RT_cutoffs[i, j] = encode_participant_df["Response_Time"].median()

In [ ]:
for index in RL_metadata.index:
  ID_index = ID_list.index(RL_metadata.at[index, "ID"])
  SI_index = list(SIs).index(RL_metadata.at[index, "SI"])

  if RL_metadata.at[index, "Response_Time"] < RT_cutoffs[ID_index, SI_index]:
    RL_metadata.at[index, "RT_Bin2"] = 1
  elif RL_metadata.at[index, "Response_Time"] > RT_cutoffs[ID_index, SI_index]:
    RL_metadata.at[index, "RT_Bin2"] = 2

In [ ]:
for index in repo_metadata.index:
  ID_index = ID_list.index(repo_metadata.at[index, "ID"])
  SI_index = list(SIs).index(repo_metadata.at[index, "SI"])

  if repo_metadata.at[index, "Response_Time"] < RT_cutoffs[ID_index, SI_index]:
    repo_metadata.at[index, "RT_Bin2"] = 1
  elif repo_metadata.at[index, "Response_Time"] > RT_cutoffs[ID_index, SI_index]:
    repo_metadata.at[index, "RT_Bin2"] = 2

In [ ]:
for index in encode_metadata.index:
  ID_index = ID_list.index(encode_metadata.at[index, "ID"])
  SI_index = list(SIs).index(encode_metadata.at[index, "SI"])

  if encode_metadata.at[index, "Response_Time"] < RT_cutoffs[ID_index, SI_index]:
    encode_metadata.at[index, "RT_Bin2"] = 1
  elif encode_metadata.at[index, "Response_Time"] > RT_cutoffs[ID_index, SI_index]:
    encode_metadata.at[index, "RT_Bin2"] = 2

In [ ]:
RL_metadata.head()

# Z-Score Gap Within Participant
Accounts for differences in gap duration across participants

In [ ]:
encode_standardized_RTs = list(np.zeros(len(ID_list)))
repo_standardized_RTs = list(np.zeros(len(ID_list)))
RL_standardized_RTs = list(np.zeros(len(ID_list)))

for i in range(0, len(ID_list)):
  ID = ID_list[i]
  encode_participant_df = encode_metadata.query("ID == @ID")
  encode_standardized_RTs[i] = np.array((encode_participant_df["Gap"] - np.nanmean(encode_participant_df["Gap"])) / np.nanstd(encode_participant_df["Gap"]))

  repo_participant_df = repo_metadata.query("ID == @ID")
  repo_standardized_RTs[i] = np.array((repo_participant_df["Gap"] - np.nanmean(repo_participant_df["Gap"])) / np.nanstd(repo_participant_df["Gap"]))

  RL_participant_df = RL_metadata.query("ID == @ID")
  RL_standardized_RTs[i] = np.array((RL_participant_df["Gap"] - np.nanmean(RL_participant_df["Gap"])) / np.nanstd(RL_participant_df["Gap"]))

encode_metadata["Gap"] = np.concatenate(encode_standardized_RTs)
repo_metadata["Gap"] = np.concatenate(repo_standardized_RTs)
RL_metadata["Gap"] = np.concatenate(RL_standardized_RTs)

# Standardize SI Variable

In [ ]:
encode_metadata.head()

In [ ]:
SIz = (np.array(SIs) - np.mean(SIs)) / np.std(SIs)
print(SIz)

In [ ]:
replace_dict = {600: SIz[0], 650: SIz[1], 700: SIz[2], 750: SIz[3], 800:SIz[4], 850:SIz[5], 900:SIz[6], 950:SIz[7], 1000:SIz[8]}

encode_metadata["SI"].replace(replace_dict, inplace = True)
repo_metadata["SI"].replace(replace_dict, inplace = True)
RL_metadata["SI"].replace(replace_dict, inplace = True)

In [ ]:
encode_metadata.head()

# Standardize Response_Time Variable

In [ ]:
encode_metadata["Response_Time"] = (encode_metadata["Response_Time"] - encode_metadata["Response_Time"].mean()) / encode_metadata["Response_Time"].std()
repo_metadata["Response_Time"] = (repo_metadata["Response_Time"] - repo_metadata["Response_Time"].mean()) / repo_metadata["Response_Time"].std()
RL_metadata["Response_Time"] = (RL_metadata["Response_Time"] - RL_metadata["Response_Time"].mean()) / RL_metadata["Response_Time"].std()

# Standardize Output Measures

In [ ]:
stan = 0

if stan:
  encode_metadata["Encode_Offset_CNV_Amp"] = (encode_metadata["Encode_Offset_CNV_Amp"] - encode_metadata["Encode_Offset_CNV_Amp"].mean()) /encode_metadata["Encode_Offset_CNV_Amp"].std()
  encode_metadata["Encode_Offset_CPP_Amp"] = (encode_metadata["Encode_Offset_CPP_Amp"] - encode_metadata["Encode_Offset_CPP_Amp"].mean()) /encode_metadata["Encode_Offset_CPP_Amp"].std()
  encode_metadata["Encode_Offset_CNV_Slope"] = (encode_metadata["Encode_Offset_CNV_Slope"] - encode_metadata["Encode_Offset_CNV_Slope"].mean()) /encode_metadata["Encode_Offset_CNV_Slope"].std()
  encode_metadata["Encode_Offset_CPP_Slope"] = (encode_metadata["Encode_Offset_CPP_Slope"] - encode_metadata["Encode_Offset_CPP_Slope"].mean()) /encode_metadata["Encode_Offset_CPP_Slope"].std()

  repo_metadata["Repo_Onset_CNV_Amp"] = (repo_metadata["Repo_Onset_CNV_Amp"] - repo_metadata["Repo_Onset_CNV_Amp"].mean()) / repo_metadata["Repo_Onset_CNV_Amp"].std()
  repo_metadata["Repo_Onset_CPP_Amp"] = (repo_metadata["Repo_Onset_CPP_Amp"] - repo_metadata["Repo_Onset_CPP_Amp"].mean()) / repo_metadata["Repo_Onset_CPP_Amp"].std()

  RL_metadata["RL_CNV_Amp"] = (RL_metadata["RL_CNV_Amp"] - RL_metadata["RL_CNV_Amp"].mean()) / RL_metadata["RL_CNV_Amp"].std()
  RL_metadata["RL_CNV_Slope"] = (RL_metadata["RL_CNV_Slope"] - RL_metadata["RL_CNV_Slope"].mean()) / RL_metadata["RL_CNV_Slope"].std()
  RL_metadata["RL_CPP_Amp"] = (RL_metadata["RL_CPP_Amp"] - RL_metadata["RL_CPP_Amp"].mean()) / RL_metadata["RL_CPP_Amp"].std()
  RL_metadata["RL_CPP_Slope"] = (RL_metadata["RL_CPP_Slope"] - RL_metadata["RL_CPP_Slope"].mean()) / RL_metadata["RL_CPP_Slope"].std()

# Save Files

In [ ]:
fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Encode.csv"
encode_metadata.to_csv(fname)

fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Repo.csv"
repo_metadata.to_csv(fname)

fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "RL.csv"
RL_metadata.to_csv(fname)

# For Checking RL analysis, remove trials with RT < 600ms

In [ ]:
#print(RL_metadata.shape)
#RL_metadata = RL_metadata[RL_metadata["Response_Time"] < 0.6]
#print(RL_metadata.shape)

# ANOVA

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "RL_CNV_Slope"

trial_numbers_RL = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = RL_metadata[RL_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    trial_numbers_RL[i, j] = condition_df.shape[0]
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "RL_CNV_Slope_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
plt.style.use("seaborn-colorblind")
palette = sns.color_palette()
colors = ["Red", "Blue"]

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 10)
plt.ylabel("CNV Slope" + " " + "$(\mu V/m^2/s)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "RL_CNV_Slope_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "RL_CNV_Amp"

trial_numbers_RL = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = RL_metadata[RL_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    trial_numbers_RL[i, j] = condition_df.shape[0]
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "RL_CNV_Amp_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.ylabel("CNV Amplitude" + " " + "$(\mu V/m^2)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "RL_CNV_Amp_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "RL_CPP_Slope"

trial_numbers_RL = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = RL_metadata[RL_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    trial_numbers_RL[i, j] = condition_df.shape[0]
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "RL_CPP_Slope_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.ylabel("CPP Slope" + " " + "$(\mu V/m^2/s)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "RL_CPP_Slope_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "RL_CPP_Amp"

trial_numbers_RL = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = RL_metadata[RL_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    trial_numbers_RL[i, j] = condition_df.shape[0]
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "RL_CPP_Amp_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.ylabel("CPP Amplitude" + " " + "$(\mu V/m^2)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "RL_CPP_Amp_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
repo_metadata.head()

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "Repo_Onset_CNV_Amp"

trial_numbers_RL = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = repo_metadata[repo_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Repo_Onset_CNV_Amp_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 10)
plt.ylabel("CNV Amplitude" + " " + "$(\mu V/m^2)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "Cue_Onset_CNV_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "Repo_Onset_CPP_Amp"

trial_numbers_RL = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = repo_metadata[repo_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Repo_Onset_CPP_Amp_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 10)
plt.ylabel("CPP Amplitude" + " " + "$(\mu V/m^2)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "Cue_Onset_CPP_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "Encode_Offset_CNV_Amp"

trial_numbers_encode = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = encode_metadata[encode_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Encode_Offset_CNV_Amp_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 10)
plt.ylabel("CNV Amplitude" + " " + "$(\mu V/m^2)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "Encode_Offset_CNV_Amp_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "Encode_Offset_CPP_Amp"

trial_numbers_encode = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = encode_metadata[encode_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Encode_Offset_CPP_Amp_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 10)
plt.ylabel("CPP Amplitude" + " " + "$(\mu V/m^2)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "Encode_Offset_CPP_Amp_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "Encode_Offset_CNV_Slope"

trial_numbers_encode = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = encode_metadata[encode_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Encode_Offset_CNV_Slope_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 10)
plt.ylabel("CNV Slope" + " " + "$(\mu V/m^2/s)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "Encode_Offset_Slope_Amp_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")

In [ ]:
labels = ["Fast", "Slow"]
conds = [1, 2]

dv = "Encode_Offset_CPP_Slope"

trial_numbers_encode = np.zeros((len(ID_list), len(conds)))

ANOVA_array = np.zeros((len(ID_list), len(labels)))

mean_RT = np.zeros((len(ID_list), len(conds)))

for i in range(0, len(ID_list)):
  participant_df = encode_metadata[encode_metadata["ID"] == ID_list[i]]

  for j in range(0, len(labels)):
    cond = conds[j]
    #condition_df = participant_df.query("RT_Bin2 == @cond")
    condition_df = participant_df[participant_df["RT_Bin2"] == cond]
    ANOVA_array[i, j] = condition_df[dv].mean()
    mean_RT[i, j] = condition_df["Response_Time"].mean()

ANOVA_df = pd.DataFrame(ANOVA_array, columns = labels, index = ID_list)
ANOVA_df

In [ ]:
if stan:
  fname = "/content/gdrive/MyDrive/Time_Reproduction/JASP_Files_TR/" + "Encode_Offset_CPP_Slope_jasp_Med_ANOVA.csv"
  ANOVA_df.to_csv(fname)

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks([0.25, 0.75])
ax.set_xticklabels(["Fast", "Slow"])

ax.plot(0.25, ANOVA_df["Fast"].mean(), marker = "o", markersize = "20", color = colors[0])
ax.plot(0.75, ANOVA_df["Slow"].mean(), marker = "o", markersize = "20", color = colors[1])
ax.plot(np.array([0.25, 0.75]), ANOVA_df.mean(axis = 0), color = "k")

for ID in ID_list:
  participant_df = ANOVA_df.loc[ID]
  ax.plot(0.25, participant_df["Fast"], marker = "o", alpha = 0.1, color = colors[0])
  ax.plot(0.75, participant_df["Slow"], marker = "o", alpha = 0.1, color = colors[1])
  ax.plot([0.25, 0.75], np.array(participant_df), alpha = 0.1, color = "k")

sns.despine()
plt.xlim(0.05, 0.95)
plt.xlabel("RT Bin", fontsize = "xx-large", labelpad = 5)
plt.ylabel("CPP Slope" + " " + "$(\mu V/m^2/s)$", fontsize = "xx-large", weight='bold')
ax = plt.gca()
ax.tick_params(labelsize = "xx-large")

fname = "/content/gdrive/MyDrive/Time_Reproduction/Plots/ERPs/" + "Encode_Offset_CPP_Slope_RT_Bins.png"
plt.savefig(fname, dpi = 300, bbox_inches = "tight")